DSCB310 – Abgabe Übungsblatt 1 (WS22/23)


Matrikelnummern:  80001, 79679 

# Stronk Data

![alt text](s2.jpg "YEAH BUDDY")

# Data Quality

In [ ]:
#imports 
import pandas as pd
import plotly.express as px
import numpy as np
import datetime
import sklearn

Fehler in der Spalte temp in der die Kommagrenze verschoben wurde

# Vorgebenen Aufgaben

# Geschäftsführung

# Abteilungsleiter